In [ ]:
import os
import pathlib

import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, clear_output

import anguilla.hypervolume as hv
from anguilla.fitness import benchmark
from anguilla.evaluation import load_logs
from anguilla.dominance import NonDominatedSet2D, NonDominatedSetKD

In [ ]:
FNS_2D = [
    'ZDT1',
    'ZDT2',
    'ZDT3',
    'ZDT4',
    'ZDT6',
    'IHR1',
    'IHR2',
    'IHR3',
    'IHR4',
    'IHR6',
    'ELLI1',
    'ELLI2',
    'CIGTAB1',
    'CIGTAB2'
]

FNS_3D = [
    'DTLZ1',
    'DTLZ2',
    'DTLZ3',
    'DTLZ4',
    'DTLZ5',
    'DTLZ6',
    'DTLZ7',
    'GELLI3',
]

In [ ]:
def compute_reference_point(logs):
    n_logs = len(logs)
    n_objectives = logs[0].data.shape[1]
    point_set = NonDominatedSet2D() if n_objectives == 2 else NonDominatedSetKD()
    for i, log in enumerate(logs):
        clear_output(wait=True)
        display(f"[{i}/{n_logs} - {i/n_logs:.2%}] {log.fn}/{log.optimizer}/{log.trial}/{log.n_evaluations} {log.data.shape}")
        point_set.insert(log.data)
    reference = point_set.upper_bound + 1.0
    return reference

def plot_hypervolume_history(logs):
    fig = plt.figure(figsize=(6, 4))
    ax = fig.add_subplot(111)
    data = {}
    key = None
    reference = compute_reference_point(logs)
    clear_output(wait=True)
    display("Reference point: ", reference)
    for log in sorted(logs, key=lambda log: (log.optimizer, log.n_evaluations, log.trial)):
        if key is None:
            key = (log.optimizer, log.n_evaluations)
        if key != (log.optimizer, log.n_evaluations):
            data[key[0]][key[1]] = np.median(data[key[0]][key[1]])
            key = (log.optimizer, log.n_evaluations)
        if log.optimizer not in data:
            data[log.optimizer] = {}
        if log.n_evaluations not in data[log.optimizer]:
            data[log.optimizer][log.n_evaluations] = []
        indicator = hv.calculate(log.data, reference, ignore_dominated=True)
        data[log.optimizer][log.n_evaluations] += [indicator]
    data[key[0]][key[1]] = np.median(data[key[0]][key[1]])
    ticks_set = False
    for opt, history in data.items():
        ax.plot(history.keys(), history.values(), marker='s', label=opt)
        if not ticks_set:
            ax.set_xticks(list(history.keys()))
            ticks_set = True
    reference_str = ",".join([f"{coord:.2E}" for coord in reference])
    ax.set_title(f"{logs[0].fn}\nReference: [{reference_str}]")
    ax.set_ylabel("Hypervolume (median)")
    ax.set_xlabel("Function evaluations")
    ax.legend()
    fig.tight_layout()
    clear_output()
    return fig

In [ ]:
for fn in FNS_2D + FNS_3D:
    display(f"Processing {fn}...")
    path = pathlib.Path("./plots/hypervolume/anguilla")
    if not path.exists():
        os.makedirs(path, exist_ok=True)
    logs = load_logs("./data/anguilla", fns=[fn], observations=["fitness"], search_subdirs=True)
    shark_logs = load_logs("./data/shark", fns=[fn], opts=['NSGAII'], observations=["fitness"], search_subdirs=False)
    fig = plot_hypervolume_history(logs + shark_logs)
    fig.savefig(path.joinpath(f"{fn}.pdf"), bbox_inches="tight")
    fig.savefig(path.joinpath(f"{fn}.png"), bbox_inches="tight")
    plt.close(fig)

In [ ]:
for fn in FNS_2D + FNS_3D:
    print(f"Processing {fn}...")
    path = pathlib.Path("./plots/hypervolume/shark")
    if not path.exists():
        os.makedirs(path, exist_ok=True)
    logs = load_logs("./data/shark", fns=[fn], observations=["fitness"], search_subdirs=False)
    fig = plot_hypervolume_history(logs)
    fig.savefig(path.joinpath(f"{fn}.pdf"), bbox_inches="tight")
    fig.savefig(path.joinpath(f"{fn}.png"), bbox_inches="tight")
    plt.close(fig)